In [16]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from matplotlib.animation import FuncAnimation, PillowWriter, FFMpegWriter
import matplotlib.colors as mcolors
import os
from IPython.display import HTML

%matplotlib inline

In [17]:
class AgentMap:
    def __init__(self, occupancy_grid, agents_data):
        """
        agents_data: 
            {'agent_id': {'width': int, 'height': int, 'positions': [time0_pos, time1_pos, ...]}}
        """
        self.occupancy_grid = occupancy_grid
        self.rows = len(occupancy_grid)
        self.cols = len(occupancy_grid[0]) if self.rows > 0 else 0
        self.agents_data = agents_data
        self.current_time = 0
        self.max_time = max(len(data['positions']) for data in agents_data.values())
        
        self.agent_patches = {}
        self.target_patches = {}

        self.fig, self.ax = plt.subplots(figsize=(12, 8))
        self.ax.set_xlim(-0.5, self.cols - 0.5)
        self.ax.set_ylim(-0.5, self.rows - 0.5)
        self.ax.set_aspect('equal')
        self.ax.invert_yaxis()  # (0,0) at upper left
        
        self.draw_grid()
        self.draw_obstacles()
        
        self.agent_patches = {}
        self.initialize_agents()
        self.initialize_targets()
        
    def draw_obstacles(self):
        """绘制障碍物"""
        for i in range(self.rows):
            for j in range(self.cols):
                if self.occupancy_grid[i][j] == 1:  
                    rect = patches.Rectangle(
                        (j - 0.5, i - 0.5), 1, 1,
                        linewidth=1, 
                        edgecolor='black',
                        facecolor='black',
                        alpha=0.8
                    )
                    self.ax.add_patch(rect)

    def draw_grid(self):
        for i in range(self.rows + 1):
            self.ax.axhline(i - 0.5, color='gray', linewidth=0.5)
        for j in range(self.cols + 1):
            self.ax.axvline(j - 0.5, color='gray', linewidth=0.5)
            
        for i in range(self.rows):
            for j in range(self.cols):
                grid_id = i * self.cols + j
                self.ax.text(j, i, str(grid_id), ha='center', va='center', 
                           fontsize=8, color='darkblue', alpha=0.7)
    
    def grid_to_coords(self, grid_id, width, height):
        row = grid_id // self.cols
        col = grid_id % self.cols
        
        x = col - 0.5
        y = row - 0.5
        
        return x, y
    
    def initialize_agents(self):
        colors = list(mcolors.TABLEAU_COLORS.values())
        
        for i, (agent_id, data) in enumerate(self.agents_data.items()):
            color = colors[i % len(colors)]
            width = data['width']
            height = data['height']
            
            initial_pos = data['positions'][0]
            x, y = self.grid_to_coords(initial_pos, width, height)
            
            rect = patches.Rectangle((x, y), width, height, 
                                   linewidth=2, edgecolor=color, 
                                   facecolor=color, alpha=0.7,
                                   label=f'Agent {agent_id}')
            self.ax.add_patch(rect)
            self.agent_patches[agent_id] = rect
            
        self.ax.legend(loc='upper right')
        self.ax.set_title(f'Agent Movement Map - Time Step: 0', fontsize=14)

    def initialize_targets(self):
        """从agent数据初始化targets（最终位置）"""
        colors = list(mcolors.TABLEAU_COLORS.values())
        
        for i, (agent_id, data) in enumerate(self.agents_data.items()):
            color = colors[i % len(colors)]
            width = data['width']
            height = data['height']
            
            # 获取最终位置（最后一个位置）
            final_pos = data['positions'][-1]
            x, y = self.grid_to_coords(final_pos, width, height)
            
            # 生成相似但不同的颜色
            target_color = self.get_similar_color(color, variation=0.2)
            
            # 创建target的矩形patch，使用虚线边框
            rect = patches.Rectangle((x, y), width, height, 
                                   linewidth=3, edgecolor=target_color, 
                                   facecolor=target_color, alpha=0.4,
                                   linestyle='--',  # 虚线边框
                                   label=f'Target {agent_id}')
            self.ax.add_patch(rect)
            self.target_patches[agent_id] = rect
            
    def get_similar_color(self, base_color, variation=0.3):
        """
        生成与基础颜色相似但不同的颜色
        base_color: 基础颜色（matplotlib颜色字符串）
        variation: 颜色变化程度（0-1）
        """
        base_rgb = mcolors.to_rgb(base_color)
        
        # 生成相似但不同的颜色
        similar_rgb = []
        for channel in base_rgb:
            # 在基础颜色基础上进行随机变化
            new_channel = max(0, min(1, channel + np.random.uniform(-variation, variation)))
            similar_rgb.append(new_channel)
        
        return similar_rgb
    

    def update_agents(self, time_step):
        for agent_id, data in self.agents_data.items():
            if time_step < len(data['positions']):
                grid_id = data['positions'][time_step]
                width = data['width']
                height = data['height']
                
                x, y = self.grid_to_coords(grid_id, width, height)
                
                rect = self.agent_patches[agent_id]
                rect.set_xy((x, y))
                rect.set_width(width)
                rect.set_height(height)
        
        self.ax.set_title(f'Agent Movement Map - Time Step: {time_step}', fontsize=14)
        return list(self.agent_patches.values())
    
    def create_animation(self, interval=500):
        def update(frame):
            return self.update_agents(frame)
        
        anim = FuncAnimation(self.fig, update, frames=self.max_time,
                           interval=interval, blit=False, repeat=True)
        return anim
    
    def export_animation(self, filename='agent_animation.gif', fps=2, dpi=100):
        anim = self.create_animation(interval=1000//fps)
        
        writer = PillowWriter(fps=fps)
        anim.save(filename, writer=writer, dpi=dpi)
        print(f"saved to: {os.path.abspath(filename)}")
        
        return anim
    
    def show_single_frame(self, time_step=0):
        self.update_agents(time_step)
        plt.show()

    def show_animation(self, interval=500):
        anim = self.create_animation(interval)
        plt.close() 
        return HTML(anim.to_jshtml())

In [18]:
occupancy_grid = []
grid_size = 8


for i in range(grid_size):
    occupancy_grid.append([0] * grid_size)

for i in range(1, grid_size - 1):
    if i != 3 and i != 4:
        occupancy_grid[i][3] = 1
        occupancy_grid[i][4] = 1
    
    
agents_data = {
        'A': {
            'width': 2,
            'height': 1,
            'positions': [0, 1, 9, 17, 18, 26, 27, 28, 29, 37, 38, 46]  
        },
        'B': {
            'width': 1,
            'height': 2,
            'positions': [6, 5, 4, 3, 2, 1, 9, 17, 25, 24, 32, 40]  
        }
}
    
agent_map = AgentMap(occupancy_grid, agents_data)
    
agent_map.export_animation('agent_movement.gif', fps=1)
agent_map.show_animation(interval=800)

saved to: /mnt/c/Users/46860/Desktop/public_graph-main/agent_movement.gif
